In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, scale, FunctionTransformer, LabelEncoder, LabelBinarizer
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.metrics import log_loss, accuracy_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.base import BaseEstimator, TransformerMixin
import xgboost as xgb

plt.style.use('ggplot')

In [ ]:

pokemon = pd.read_csv('../input/pokemon.csv')
combats = pd.read_csv('../input/combats.csv')


test = pd.read_csv('../input/tests.csv')

In [ ]:
pokemon.describe()

In [ ]:
pokemon_new = pokemon.dropna(subset = ['Name'])
pokemon_new.drop(['#'], axis = 1, inplace = True)
pokemon_new.info()

In [ ]:
combats_namen = combats.replace(pokemon.Name)
combats_namen = combats_namen.dropna()
combats_namen.info()

In [ ]:
combats_namen.Winner[combats_namen.Winner == combats_namen.First_pokemon] = 0
combats_namen.Winner[combats_namen.Winner == combats_namen.Second_pokemon] = 1
combats_namen.head()

In [ ]:

#combats_first = pd.concat([combats_namen, pokemon_new.rename(columns = {'Name': 'First_pokemon'})], axis = 1, ignore_index = False, sort = False)
combats_first = pd.merge(combats_namen, pokemon_new, how = 'inner', left_on = ['First_pokemon'], right_on = ['Name'])



combats_second = pd.merge(combats_namen, pokemon_new, how = 'inner', left_on = ['Second_pokemon'], right_on = ['Name'])



#combats_final = pd.merge(combats_first, combats_second, how = 'inner', left_on = ['First_pokemon', 'Second_pokemon', 'Winner'], 
 #                    right_on = ['First_pokemon', 'Second_pokemon', 'Winner'] )

combats_final = pd.merge(combats_first, combats_second, how = 'inner', left_index = True, right_index = True )

combats_final.info()

In [ ]:
combats_final['Diff_HP'] = combats_final['HP_x'] - combats_final['HP_y']
combats_final['Diff_Attack'] = combats_final['Attack_x'] - combats_final['Attack_y']
combats_final['Diff_Defense'] = combats_final['Defense_x'] - combats_final['Defense_y']
combats_final['Diff_Sp_Attack'] = combats_final['Sp. Atk_x'] - combats_final['Sp. Atk_y']
combats_final['Diff_Sp_Defense'] = combats_final['Sp. Def_x'] - combats_final['Sp. Def_y']
combats_final['Diff_Speed'] = combats_final['Speed_x'] - combats_final['Speed_y']

                                        

combats_final.rename(columns = {'First_pokemon_x': 'First_pokemon', 'Second_pokemon_x':'Second_pokemon', 'Winner_x': 'Winner', 'Type 1_x':'Type_1_First', 
                               'Type 2_x': 'Type_2_First', 'Type 1_y':'Type_1_Second','Type 2_y': 'Type_2_Second', 'Generation_x':'Generation_First', 
                                'Generation_y':'Generation_Second', 'Legendary_x': 'Legendary_First', 'Legendary_y':'Legendary_Second'}, inplace = True)

cols_to_keep = ['Diff_HP', 'Diff_Attack', 'Diff_Defense', 'Diff_Sp_Attack','Diff_Sp_Defense', 'Diff_Speed', 'First_pokemon' ,'Second_pokemon',
               'Winner',  'Generation_First', 'Generation_Second', 'Legendary_First', 'Legendary_Second']

combats_final = combats_final[cols_to_keep]



combats_final.info()

In [ ]:
combats_final.fillna('Unknown', inplace = True)

combats_final.info()

In [ ]:
combats_final.drop(['Legendary_First', 'Legendary_Second', 'First_pokemon', 'Second_pokemon'], axis = 1,  inplace = True)

combats_final['Generation_First'] = str(combats_final['Generation_First'])
combats_final['Generation_Second'] = str(combats_final['Generation_Second'])
combats_final.info()

In [ ]:
#scale data
scaler = StandardScaler()

cols_to_scale = list(combats_final.select_dtypes(include=[np.number]).columns.values)
combats_final[['Diff_HP','Diff_Attack','Diff_Defense','Diff_Sp_Attack','Diff_Sp_Defense','Diff_Speed']]= scaler.fit_transform(combats_final[['Diff_HP','Diff_Attack','Diff_Defense','Diff_Sp_Attack','Diff_Sp_Defense','Diff_Speed']])

In [ ]:
#categorical naar dummies
to_encode = ['Generation_First','Generation_Second']

combats_final = pd.concat([pd.get_dummies(combats_final[to_encode], drop_first = True), combats_final[['Diff_HP','Diff_Attack','Diff_Defense','Diff_Sp_Attack','Diff_Sp_Defense','Diff_Speed', 'Winner']]], axis = 1)

to_encode = combats_final.select_dtypes(include=[object]).columns
combats_final = pd.get_dummies(combats_final, columns=to_encode, drop_first=True)


combats_final.info()

In [ ]:

y = combats_final['Winner']
X = combats_final.drop('Winner', axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X,y,  
                                                    random_state=2018, stratify= y)

# hyperparameter grid
param_grid = {
    'n_estimators': [500,1000],
    'max_features': ['auto', 'sqrt']
}
#RandomForest classifier
rfc = RandomForestClassifier(random_state = 2018)
#GridSearchCV
rf_cv = GridSearchCV(rfc, param_grid = param_grid, cv=10, n_jobs = -1)
rf_cv.fit(X_train, y_train)

# Print the optimal parameters and best score
print("Tuned Randomforest Parameter: {}".format(rf_cv.best_params_))
print("Tuned Randomforest Accuracy: {}".format(rf_cv.best_score_))

y_pred_rf = rf_cv.predict(X_test)
print(confusion_matrix(y_test, y_pred_rf))

In [ ]:
#xgb_params = {'max_depth': [5, 9], 
           #   'learning_rate': [0.001, 0.01, 0.05],
           #  'colsample_bytree' : [0.1, 0.5, 1]}

xgb_params = {'colsample_bytree': [0.1, 0.5, 1], 'learning_rate': [0.05], 'max_depth': [9]}


model_xgb = xgb.XGBClassifier(seed=1, n_estimators=1000)
grid = GridSearchCV(model_xgb, xgb_params, cv=5 ,scoring='accuracy', verbose = 1, n_jobs= -1)

grid.fit(X_train,y_train)



print("Best score: %.1f%%" % (grid.best_score_*100))
print("Best parameter set: %s" % (grid.best_params_))



In [ ]:
y_pred = grid.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.1f%%" % (accuracy * 100.0))

In [ ]:
#xgb_params = {'max_depth': [3, 6, 9], 
#                              'learning_rate': [0.05]}


#model_xgb = xgb.XGBClassifier(seed=1, n_estimators=1000)
#grid = GridSearchCV(model_xgb, xgb_params, cv=5 ,scoring='accuracy', verbose = 1, n_jobs= -1)

In [ ]:
#num_cols = list(combats_final.select_dtypes(include=[np.number]).columns.values)
#object_cols = list(combats_final.select_dtypes(include=[np.object]).columns.values)

#get_object_data = FunctionTransformer(lambda x: x[['First_pokemon','Second_pokemon','Type_1_First','Type_2_First','Type_1_Second','Type_2_Second', 
#                                                   'Generation_First', 'Generation_Second']])

#get_numeric_data = FunctionTransformer(lambda x: x[['Diff_HP','Diff_Attack','Diff_Defense','Diff_Sp_Attack','Diff_Sp_Defense',
#                                                    'Diff_Speed']])

In [ ]:
test.describe()

In [ ]:
test_namen = test.replace(pokemon.Name)
test_namen = test_namen.dropna()

#test_first = pd.concat([test_namen, pokemon_new.rename(columns = {'Name': 'First_pokemon'})], axis = 1, ignore_index = False, sort = False)
test_first = pd.merge(test_namen, pokemon_new, how = 'inner', left_on = ['First_pokemon'], right_on = ['Name'])



test_second = pd.merge(test_namen, pokemon_new, how = 'inner', left_on = ['Second_pokemon'], right_on = ['Name'])



test_final = pd.merge(test_first, test_second, how = 'inner', left_index = True, right_index = True )

test_final['Diff_HP'] = test_final['HP_x'] - test_final['HP_y']
test_final['Diff_Attack'] = test_final['Attack_x'] - test_final['Attack_y']
test_final['Diff_Defense'] = test_final['Defense_x'] - test_final['Defense_y']
test_final['Diff_Sp_Attack'] = test_final['Sp. Atk_x'] - test_final['Sp. Atk_y']
test_final['Diff_Sp_Defense'] = test_final['Sp. Def_x'] - test_final['Sp. Def_y']
test_final['Diff_Speed'] = test_final['Speed_x'] - test_final['Speed_y']

                                        

test_final.rename(columns = {'First_pokemon_x': 'First_pokemon', 'Second_pokemon_x':'Second_pokemon', 'Type 1_x':'Type_1_First', 
                               'Type 2_x': 'Type_2_First', 'Type 1_y':'Type_1_Second','Type 2_y': 'Type_2_Second', 'Generation_x':'Generation_First', 
                                'Generation_y':'Generation_Second', 'Legendary_x': 'Legendary_First', 'Legendary_y':'Legendary_Second'}, inplace = True)

cols_to_keep = ['Diff_HP', 'Diff_Attack', 'Diff_Defense', 'Diff_Sp_Attack','Diff_Sp_Defense', 'Diff_Speed', 'First_pokemon' ,'Second_pokemon',
                 'Generation_First', 'Generation_Second', 'Legendary_First', 'Legendary_Second']

test_final = test_final[cols_to_keep]

test_final.fillna('Unknown', inplace = True)


test_final.drop(['Legendary_First', 'Legendary_Second', 'First_pokemon', 'Second_pokemon'], axis = 1,  inplace = True)

test_final['Generation_First'] = str(test_final['Generation_First'])
test_final['Generation_Second'] = str(test_final['Generation_Second'])

cols_to_scale = list(test_final.select_dtypes(include=[np.number]).columns.values)
test_final[['Diff_HP','Diff_Attack','Diff_Defense','Diff_Sp_Attack','Diff_Sp_Defense','Diff_Speed']]= scaler.fit_transform(test_final[['Diff_HP','Diff_Attack','Diff_Defense','Diff_Sp_Attack','Diff_Sp_Defense','Diff_Speed']])

#categorical naar dummies
to_encode = ['Generation_First','Generation_Second']

test_final = pd.concat([pd.get_dummies(test_final[to_encode], drop_first = True), test_final[['Diff_HP','Diff_Attack','Diff_Defense','Diff_Sp_Attack','Diff_Sp_Defense','Diff_Speed']]], axis = 1)



In [ ]:
test_final.info()


In [ ]:
test_final['Prediction'] = grid.predict(test_final)

In [ ]:
test_final.head()



class Columns(BaseEstimator, TransformerMixin):
    def __init__(self, names=None):
        self.names = names

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X):
        return X[self.names]

numeric = ['First_pokemon','Second_pokemon','Type_1_First','Type_2_First','Type_1_Second','Type_2_Second', 
                                                   'Generation_First', 'Generation_Second']
categorical = ['Diff_HP','Diff_Attack','Diff_Defense','Diff_Sp_Attack','Diff_Sp_Defense',
                                                    'Diff_Speed']
# hyperparameter grid
param_grid = {
    'n_estimators': [500,1000],
    'max_features': ['auto', 'sqrt']
}


pipe = Pipeline([
    ("features", FeatureUnion([
        ('numeric', make_pipeline(Columns(names=numeric),StandardScaler())),
        ('categorical', make_pipeline(Columns(names=categorical),LabelBinarizer()))
    ])),
    ('model', GridSearchCV(RandomForestClassifier(random_state = 2018), param_grid = param_grid, cv=10, n_jobs = -1))
])

process_and_join_features = FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                    ('scaler', StandardScaler())
                ])),
                ('object_features', Pipeline([
                    ('selector', get_object_data),
                    ('encoder', LabelBinarizer())
                ]))
             ]
        )



# Instantiate nested pipeline: pl
pl = Pipeline([
        ('union', process_and_join_features),
        ('clf', LogisticRegression())
    ])


# Fit pl to the training data
pipe.fit_transform(X_train, y_train)

# Compute and print accuracy
accuracy = pipe.score(X_test, y_test)
print("\nAccuracy on sample data - all data: ", accuracy)